In [1]:
from pyspark.ml.classification import RandomForestClassifier, RandomForestClassificationModel

import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext(appName="RFTEST")
from pyspark.sql.session import SparkSession
spark = SparkSession(sc)

22/05/04 19:58:04 WARN Utils: Your hostname, herex resolves to a loopback address: 127.0.1.1; using 10.245.4.22 instead (on interface wlp58s0)
22/05/04 19:58:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/04 19:58:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Cargamos el modelo RandomForest guardado

In [2]:
rf_model = RandomForestClassificationModel.load("./Model_RF_V1")

### Creamos un titulo para hacer la prueba

In [3]:
import pandas as pd
title = "Crypto is the best"

df = spark.createDataFrame([(title,)], ["news"])

### Tokenizamos

In [4]:
import pandas as pd
from pyspark.ml.feature import Tokenizer
tokenization=Tokenizer(inputCol='news',outputCol='tokens')
tokenized_df=tokenization.transform(df)
pd.DataFrame(tokenized_df.take(1), columns=tokenized_df.columns).transpose()


,0
news,Crypto is the best
tokens,"[crypto, is, the, best]"


### Quitamos las StopWords

In [5]:
from pyspark.ml.feature import StopWordsRemover
stopword_removal=StopWordsRemover(inputCol='tokens',outputCol='refined_tokens')
refined_df=stopword_removal.transform(tokenized_df)
#pd.DataFrame(refined_df.take(1), columns=refined_df.columns).transpose()

### Extraemos las Features

In [6]:
from pyspark.ml.feature import HashingTF, IDF

hashingTF = HashingTF(inputCol="refined_tokens", outputCol="rawFeatures", numFeatures=20)
featurizedData = hashingTF.transform(refined_df)

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

#pd.DataFrame(rescaledData.take(5), columns=rescaledData.columns).transpose()

### Aplicamos el modelo que hemos cargado

In [7]:
predictions = rf_model.transform(rescaledData)
pd.set_option('display.max_colwidth', None)
pd.DataFrame(predictions.take(1), columns=predictions.columns).transpose()

,0
news,Crypto is the best
tokens,"[crypto, is, the, best]"
refined_tokens,"[crypto, best]"
rawFeatures,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)"
features,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)"
rawPrediction,"[1.8121143926702563, 11.997208083581125, 6.190677523748616]"
probability,"[0.09060571963351283, 0.5998604041790564, 0.30953387618743083]"
prediction,1.0
